In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import pymongo

In [ ]:
states = [  "Abia",
            "Abuja",
            "Adamawa",
            "Akwa-Ibom",
            "Anambra",
            "Bauchi",
            "Bayelsa",
            "Benue",
            "Borno",
            "Cross-River",
            "Delta",
            "Ebonyi",
            "Edo",
            "Ekiti",
            "Enugu",
            "Gombe",
            "Imo",
            "Jigawa",
            "Kaduna",
            "Kano",
            "Katsina",
            "Kebbi",
            "Kogi",
            "Kwara",
            "Lagos",
            "Nassarawa",
            "Niger",
            "Ogun",
            "Ondo",
            "Osun",
            "Oyo",
            "Plateau",
            "Rivers",
            "Sokoto",
            "Taraba",
            "Yobe",
            "Zamfara"
            ]

In [ ]:
data = {}
for state in states:
    print(state)
    req = Request("https://nigeria.opendataforafrica.org/apps/atlas/{}".format(state))
    state_html_page = urlopen(req)
    soup = BeautifulSoup(state_html_page, "lxml")
    links = []
    for link in soup.findAll("a"):
        links.append(link.get("href"))

    state_1 = state+"/"
    state_info = list()

    if "-" in state:
        state.replace("-","")

    for link in links:
        if state_1 in link:
            ppty = link.split("/")[-1].replace("-","_").lower()
            info={}
            state_info_html_page = urlopen(link)
            soup = BeautifulSoup(state_info_html_page,'lxml')
            val = list()
            for tr in soup.findAll('tr'):
                vals = tr.get_text('', strip=False)
                vals =  vals.replace('%','').replace('Change,\xa0','Change'
                        ).replace('\n\n','\nnan').strip('\n').split('\n')
                val.append(vals)
            info[ppty] = val
            state_info.append(info)
        else:
            continue
    data[state] = state_info

In [ ]:
## connect to mongodb server
## NB. make sure server is already running
my_client = pymongo.MongoClient("mongodb://localhost:27017/local")

collection_name = "opendataforafrica"

db = my_client[collection_name]
my_collection = db[collection_name]

## insert reviews into collection
my_collection.insert_one(data)